<a href="https://colab.research.google.com/github/mihirkawatra/cifar10-densenet/blob/master/DNST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam, SGD
import math

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 500
num_classes = 10
epochs = 100
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [49]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 12)   324         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 12)   48          conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 12)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [0]:
from keras.preprocessing.image import ImageDataGenerator
image_generator = ImageDataGenerator(
            rotation_range=10,
            zoom_range = 0.05, 
            width_shift_range=0.07,
            height_shift_range=0.07,
            horizontal_flip=True,
            vertical_flip=False, 
            data_format="channels_last")
        # fit data for zca whitening
image_generator.fit(x_train, augment=True)

In [0]:
from keras.callbacks import ReduceLROnPlateau, CSVLogger,EarlyStopping,ModelCheckpoint
import numpy as np
# lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
csv_logger = CSVLogger('DNST_augmented_model_lr_drop_decay.csv')
early_stopper = EarlyStopping(monitor='val_loss',min_delta=0.01,patience=5)
model_checkpoint = ModelCheckpoint('DNST_augmented_model_lr_drop_decay.h5',monitor = 'val_loss', verbose = 1,save_best_only=True)

In [0]:
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

In [0]:
from keras.callbacks import LearningRateScheduler
def step_decay(epoch):
	initial_lrate = 0.5
	drop = 0.5
	epochs_drop = 10.0
	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lrate
lrate = LearningRateScheduler(step_decay)
sgd = SGD(lr=0.0, momentum=0.9, decay=0.0, nesterov=False)

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [54]:
model.fit_generator(image_generator.flow(x_train, y_train, batch_size=batch_size),
                        epochs=60,
                        steps_per_epoch=len(x_train) / batch_size,
#                         validation_data=val_gen.flow(x_test, y_test),
#                         validation_steps = len(x_test) / batch_size,
                        validation_data=(x_test,y_test),
                        callbacks=[csv_logger,model_checkpoint,lrate],
                        verbose=1)

Epoch 1/60
391/390 [==============================] - 129s 329ms/step - loss: 1.3161 - acc: 0.5266 - val_loss: 2.3372 - val_acc: 0.3744

Epoch 00001: val_loss improved from inf to 2.33718, saving model to DNST_augmented_model_lr_drop_decay.h5
Epoch 2/60
391/390 [==============================] - 109s 280ms/step - loss: 1.1018 - acc: 0.6103 - val_loss: 3.4936 - val_acc: 0.4088

Epoch 00002: val_loss did not improve from 2.33718
Epoch 3/60
391/390 [==============================] - 109s 280ms/step - loss: 1.0205 - acc: 0.6409 - val_loss: 1.4669 - val_acc: 0.6185

Epoch 00003: val_loss improved from 2.33718 to 1.46693, saving model to DNST_augmented_model_lr_drop_decay.h5
Epoch 4/60
391/390 [==============================] - 110s 280ms/step - loss: 0.9753 - acc: 0.6570 - val_loss: 1.4086 - val_acc: 0.5854

Epoch 00004: val_loss improved from 1.46693 to 1.40861, saving model to DNST_augmented_model_lr_drop_decay.h5
Epoch 5/60
391/390 [==============================] - 109s 279ms/step - los

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

In [55]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 8s 775us/step
Test loss: 0.6036345655441284
Test accuracy: 0.8028


In [0]:
from google.colab import files

files.download('DNST_augmented_model.h5')